In [ ]:
import os
import time
import json
import openai
import pandas as pd
from tqdm import tqdm
from secret import OPENAI_API_KEY

import regex  # instead of re

REQUESTING_PROMPT = """Please extract and score this MR study based on the rubric provided in the uploaded file.

**Output Format:**  
Return only a single JSON object with the following structure:  

{
  "study_id": "Smith2022",  
  "scoring": {  
    "A1": "1",  
    "A2a": "0",  
    "A2b": "1",  
    ... (all rubric items A1–A12, B1–B12 with values 1, 0, N/A, or 'Not assessed')  
  },  
  "evidence": {  
    "A1": {  
      "score": "1",  
      "quote": "Exact quoted evidence from the paper supporting A1 score.",  
      "section": "Methods - Data Sources"  
    },  
    "A2a": {  
      "score": "0",  
      "quote": "Not reported",  
      "section": "Not reported"  
    },  
    ... (repeat for all rubric items A1–A12, B1–B12)  
  }  
}

**Important:**  
- Ensure the JSON is valid and can be parsed directly.  
- Do not include any additional text, explanations, or formatting outside of the JSON object.  
- For missing or unclear information, set the score to `"Not assessed"` and set `"quote"` and `"section"` to `"Not reported"`.  

Use the citation shorthand (first author + year, e.g., Smith2022) as the `study_id`.
"""

def extract_json_objects(text):
    # Try to load full string as a list of dicts
    try:
        data = json.loads(text)
        if isinstance(data, list):
            return data[0]
        else:
            return data
    except json.JSONDecodeError:
        print("Response:")  # Print first 500 chars for debugging
        print(text[:500])
        pass  # Try regex fallback below

    # Fallback regex in case it's not clean JSON
    pattern = r"\{(?:[^{}]|(?R))*\}"  # recursively match nested {}
    matches = regex.findall(pattern, text)

    objects = []
    for m in matches:
        try:
            obj = json.loads(m)
            objects.append(obj)
        except json.JSONDecodeError:
            continue

    return objects[0] if objects else None


# Set your OpenAI API key
openai.api_key = OPENAI_API_KEY  # Ensure this environment variable is set

# Paths
pdf_dir = "./documents"
output_csv = "chatgpt_new_extracted_mr_data.csv"

# Initialize OpenAI client
client = openai.OpenAI(
    api_key=openai.api_key,
)

# Upload the Word file (rubric)
with open("./Prompt_Scoring (ChatGPT) - Richard.docx", "rb") as f:
    rubric_file = client.files.create(file=f, purpose="assistants")

# Create assistant WITHOUT attaching rubric file
assistant = client.beta.assistants.create(
    name="MR Study Extractor",
    instructions="You are a genetic epidemiologist and Methods Audit reviewer. Refer to the uploaded rubric for all scoring and extraction instructions.",
    model="gpt-4-turbo",
    tools=[{"type": "file_search"}],
)

# Create directory for responses
RESPONSE_DIR = "./responses"
os.makedirs(RESPONSE_DIR, exist_ok=True)


class GlobalDebug:
    pass

import random

def retry_with_backoff(func, *args, **kwargs):
    max_retries = 5
    delay = 10  # Start with 10 seconds
    for attempt in range(max_retries):
        try:
            return func(*args, **kwargs)
        except openai.RateLimitError as e:
            wait_time = getattr(e, "retry_after", delay)
            print(f"⚠️ Rate limit hit: retrying in {wait_time:.2f}s...")
            time.sleep(wait_time + random.uniform(0, 2))  # jitter
            delay = min(delay * 2, 60)  # exponential backoff, max 60s
        except openai.APIError as e:
            print(f"⚠️ API error: {e}. Retrying in {delay}s...")
            time.sleep(delay + random.uniform(0, 2))
            delay = min(delay * 2, 60)
        except Exception as e:
            print(f"❌ Unexpected error: {e}")
            raise
    raise RuntimeError("❌ Max retries exceeded.")


def process_pdf(file_path, index):
    file_name = os.path.basename(file_path)
    print(f"\n🔍 Processing [{index}] {file_name}")

    # Upload the PDF
    try:
        with open(file_path, "rb") as f:
            uploaded_pdf = client.files.create(file=f, purpose="assistants")
    except Exception as e:
        print(f"[ERROR][{index}] Failed to upload file {file_name}:\n{e}")
        return None

    # Create a thread
    try:
        thread = client.beta.threads.create()
    except Exception as e:
        print(f"[ERROR][{index}] Failed to create thread for {file_name}:\n{e}")
        return None

    def run_prompt(prompt, label):
        retry_with_backoff(
            client.beta.threads.messages.create,
            thread_id=thread.id,
            role="user",
            content=prompt,
            attachments=[
                {"file_id": uploaded_pdf.id, "tools": [{"type": "file_search"}]},
                {"file_id": rubric_file.id, "tools": [{"type": "file_search"}]},
            ],
        )

        run = retry_with_backoff(
            client.beta.threads.runs.create,
            thread_id=thread.id,
            assistant_id=assistant.id,
        )

        while True:
            run_status = client.beta.threads.runs.retrieve(
                thread_id=thread.id, run_id=run.id
            )

            GlobalDebug.run_status = run_status
            GlobalDebug.thread_id = thread.id
            GlobalDebug.run_id = run.id

            if run_status.status == "completed":
                break
            elif run_status.status == "failed":
                print(f"Status info: {run_status}")
                raise RuntimeError(f"Run failed on {label}")
            time.sleep(1)

        messages = client.beta.threads.messages.list(thread_id=thread.id)
        response_text = messages.data[0].content[0].text.value

        # Save raw response
        raw_path = os.path.join(RESPONSE_DIR, f"{index:03d}_{file_name}_{label}.txt")
        with open(raw_path, "w", encoding="utf-8") as f:
            f.write(response_text)

        print(response_text)

        return extract_json_objects(response_text)

    # Run prompt and get data
    data = run_prompt(
        REQUESTING_PROMPT,
        "scoring",
    )

    if not data:
        print(f"[SKIP][{index}] Skipping {file_name} due to failed prompt.")
        return None

    combined = {"study_id": file_name, **data}
    return combined


# Process all PDFs in the directory
results = []
pdf_files = sorted([f for f in os.listdir(pdf_dir) if f.endswith(".pdf")])

for i, pdf_file in enumerate(tqdm(pdf_files, desc="Processing PDFs"), 1):
    pdf_path = os.path.join(pdf_dir, pdf_file)
    result = process_pdf(pdf_path, i)
    if result:
        results.append(result)
    time.sleep(90)

# Save results to CSV
if results:
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"\n✅ Saved extracted data to: {output_csv}")
else:
    print("\n⚠️ No data extracted.")

Processing PDFs:   0%|          | 0/3 [00:00<?, ?it/s]


🔍 Processing [1] #1 - Ahn 2020.pdf
{
  "study_id": "Ahn2020",
  "scoring": {
    "A1": "0",
    "A2a": "Not assessed",
    "A2b": "Not assessed",
    "A2c": "Not assessed",
    "A3a": "1",
    "A3b": "1",
    "A4": "1",
    "A5": "0",
    "A6": "Not assessed",
    "A7": "Not assessed",
    "A8": "Not assessed",
    "A9": "1",
    "A10a": "1",
    "A10b": "1",
    "A11a": "1",
    "A11b": "1",
    "A12": "0",
    "B1a": "1",
    "B1b": "Not assessed",
    "B2": "Not assessed",
    "B3a": "1",
    "B3b": "Not assessed",
    "B4": "Not assessed",
    "B5": "Not assessed",
    "B6": "Not assessed",
    "B7a": "Not assessed",
    "B7b": "Not assessed",
    "B8a": "Not assessed",
    "B8b": "Not assessed",
    "B9": "Not assessed",
    "B10": "Not assessed",
    "B11": "Not assessed"
  },
  "evidence": {
    "A1": {
      "score": "0",
      "quote": "Missing source, ancestry, and sample size for either exposure GWAS or outcome GWAS",
      "section": "Abstract and Methods"
    },
    "A2a"

Processing PDFs:  33%|███▎      | 1/3 [02:29<04:59, 149.88s/it]


🔍 Processing [2] #3 - Aoki 2022.pdf
{
  "study_id": "Aoki2022",
  "scoring": {
    "A1": "Not assessed",
    "A2a": "Not assessed",
    "A2b": "Not assessed",
    "A2c": "Not assessed",
    "A3a": "Not assessed",
    "A3b": "Not assessed",
    "A4": "Not assessed",
    "A5": "Not assessed",
    "A6": "Not assessed",
    "A7": "Not assessed",
    "A8": "Not assessed",
    "A9": "Not assessed",
    "A10a": "Not assessed",
    "A10b": "Not assessed",
    "A11a": "Not assessed",
    "A11b": "Not assessed",
    "A12": "Not assessed",
    "B1a": "Not assessed",
    "B1b": "Not assessed",
    "B2": "Not assessed",
    "B3a": "Not assessed",
    "B3b": "Not assessed",
    "B4": "Not assessed",
    "B5": "Not assessed",
    "B6": "Not assessed",
    "B7a": "Not assessed",
    "B7b": "Not assessed",
    "B8a": "Not assessed",
    "B8b": "Not assessed",
    "B9": "Not assessed",
    "B10": "Not assessed",
    "B11": "Not assessed"
  },
  "evidence": {
    "A1": {
      "score": "Not assessed",
 

Processing PDFs:  67%|██████▋   | 2/3 [05:07<02:34, 154.70s/it]


🔍 Processing [3] #4 - AuYeung 2023.pdf
{
  "study_id": "AuYeung2023",
  "scoring": {
    "A1": "1",
    "A2a": "1",
    "A2b": "1",
    "A2c": "1",
    "A3a": "1",
    "A3b": "1",
    "A4": "1",
    "A5": "1",
    "A6": "1",
    "A7": "Not assessed",
    "A8": "1",
    "A9": "1",
    "A10a": "1",
    "A10b": "1",
    "A11a": "1",
    "A11b": "1",
    "A12": "1",
    "B1a": "1",
    "B1b": "1",
    "B2": "1",
    "B3a": "1",
    "B3b": "Not assessed",
    "B4": "Not assessed",
    "B5": "1",
    "B6": "1",
    "B7a": "1",
    "B7b": "Not assessed",
    "B8a": "1",
    "B8b": "1",
    "B9": "Not assessed",
    "B10": "Not assessed",
    "B11": "1"
  },
  "evidence": {
    "A1": {
      "score": "1",
      "quote": "\"We extracted strong (P-value< 5×10–8) and independent (r2<0.001) genetic predictors of ALT and AST from Key Messages\"",
      "section": "Methods - Data Sources"
    },
    "A2a": {
      "score": "1",
      "quote": "\"We extracted strong (P-value< 5×10–8) and independent

Processing PDFs: 100%|██████████| 3/3 [07:42<00:00, 154.29s/it]


✅ Saved extracted data to: chatgpt_new_extracted_mr_data.csv


In [2]:
df

,study_id,scoring,evidence
0,Ahn2020,"{'A1': '0', 'A2a': 'Not assessed', 'A2b': 'Not...","{'A1': {'score': '0', 'quote': 'Missing source..."
1,Aoki2022,"{'A1': 'Not assessed', 'A2a': 'Not assessed', ...","{'A1': {'score': 'Not assessed', 'quote': 'Not..."
2,AuYeung2023,"{'A1': '1', 'A2a': '1', 'A2b': '1', 'A2c': '1'...","{'A1': {'score': '1', 'quote': '""We extracted ..."
